In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import style
import random 
import nltk
#!pip install tweet-preprocessor
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from tensorflow.keras.optimizers import Adam

import numpy as np
from numpy import array
import pandas as pd

from sklearn.model_selection import train_test_split

In [5]:
ds= pd.read_csv('/content/drive/MyDrive/TripAdvisor.csv')
ds.head()

,Unnamed: 0,Review,Rating,Senti
0,0,nice hotel expensive parking got good deal sta...,4,1
1,1,ok nothing special charge diamond member hilto...,2,0
2,2,nice rooms not 4* experience hotel monaco seat...,3,1
3,3,"unique, great stay, wonderful time hotel monac...",5,1
4,4,"great stay great stay, went seahawk game aweso...",5,1


In [6]:
ds.head()

,Unnamed: 0,Review,Rating,Senti
0,0,nice hotel expensive parking got good deal sta...,4,1
1,1,ok nothing special charge diamond member hilto...,2,0
2,2,nice rooms not 4* experience hotel monaco seat...,3,1
3,3,"unique, great stay, wonderful time hotel monac...",5,1
4,4,"great stay great stay, went seahawk game aweso...",5,1


In [7]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [8]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [9]:
ds['Review']=ds['Review'].str.replace('[^\w\s]',"")
ds['Review']

0        nice hotel expensive parking got good deal sta...
1        ok nothing special charge diamond member hilto...
2        nice rooms not 4 experience hotel monaco seatt...
3        unique great stay wonderful time hotel monaco ...
4        great stay great stay went seahawk game awesom...
                               ...                        
20486    best kept secret 3rd time staying charm not 5s...
20487    great location price view hotel great quick pl...
20488    ok just looks nice modern outside desk staff n...
20489    hotel theft ruined vacation hotel opened sept ...
20490    people talking ca nt believe excellent ratings...
Name: Review, Length: 20491, dtype: object

In [10]:
ds['Review']=ds['Review'].str.replace('[^a-zA-Z#]'," ")
ds['Review']


0        nice hotel expensive parking got good deal sta...
1        ok nothing special charge diamond member hilto...
2        nice rooms not   experience hotel monaco seatt...
3        unique great stay wonderful time hotel monaco ...
4        great stay great stay went seahawk game awesom...
                               ...                        
20486    best kept secret  rd time staying charm not  s...
20487    great location price view hotel great quick pl...
20488    ok just looks nice modern outside desk staff n...
20489    hotel theft ruined vacation hotel opened sept ...
20490    people talking ca nt believe excellent ratings...
Name: Review, Length: 20491, dtype: object

In [11]:
ds['Review']=ds['Review'].apply(lambda x : " ".join ([x for x in x.split() if len(x)>3]))
ds['Review']

0        nice hotel expensive parking good deal stay ho...
1        nothing special charge diamond member hilton d...
2        nice rooms experience hotel monaco seattle goo...
3        unique great stay wonderful time hotel monaco ...
4        great stay great stay went seahawk game awesom...
                               ...                        
20486    best kept secret time staying charm star beat ...
20487    great location price view hotel great quick pl...
20488    just looks nice modern outside desk staff part...
20489    hotel theft ruined vacation hotel opened sept ...
20490    people talking believe excellent ratings hotel...
Name: Review, Length: 20491, dtype: object

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop=stopwords.words('english')
ds['Review']=ds['Review'].apply(lambda x : " ".join (x for x in x.split() if x not in stop))

ds['Review']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        nice hotel expensive parking good deal stay ho...
1        nothing special charge diamond member hilton d...
2        nice rooms experience hotel monaco seattle goo...
3        unique great stay wonderful time hotel monaco ...
4        great stay great stay went seahawk game awesom...
                               ...                        
20486    best kept secret time staying charm star beat ...
20487    great location price view hotel great quick pl...
20488    looks nice modern outside desk staff particula...
20489    hotel theft ruined vacation hotel opened sept ...
20490    people talking believe excellent ratings hotel...
Name: Review, Length: 20491, dtype: object

In [13]:
from textblob import TextBlob
from textblob import Word

ds['Review']=ds['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

ds['Review']

0        nice hotel expensive parking good deal stay ho...
1        nothing special charge diamond member hilton d...
2        nice room experience hotel monaco seattle good...
3        unique great stay wonderful time hotel monaco ...
4        great stay great stay went seahawk game awesom...
                               ...                        
20486    best kept secret time staying charm star beat ...
20487    great location price view hotel great quick pl...
20488    look nice modern outside desk staff particular...
20489    hotel theft ruined vacation hotel opened sept ...
20490    people talking believe excellent rating hotel ...
Name: Review, Length: 20491, dtype: object

In [14]:
text = ds['Review'].tolist()
text[:3]

['nice hotel expensive parking good deal stay hotel anniversary arrived late evening took advice previous review valet parking check quick easy little disappointed nonexistent view room room clean nice size comfortable woke stiff neck high pillow soundproof like heard music room night morning loud bang door opening closing hear people talking hallway maybe noisy neighbor aveda bath product nice goldfish stay nice touch taken advantage staying longer location great walking distance shopping overall nice experience parking night',
 'nothing special charge diamond member hilton decided chain shot anniversary seattle start booked suite paid extra website description suite bedroom bathroom standard hotel room took printed reservation desk showed said thing like couch desk clerk told mixed suite description kimpton website sorry free breakfast kidding embassy suit sitting room bathroom bedroom unlike kimpton call suite stay offer correct false advertising send kimpton preferred guest website

In [15]:
y = ds['Senti']

In [16]:
token = Tokenizer()
token.fit_on_texts(text)

In [17]:
vocab_size  = len(token.word_index) + 1
vocab_size

69003

In [18]:
encoded_text = token.texts_to_sequences(text)

In [19]:
print(encoded_text[:3])

[[9, 1, 151, 191, 4, 200, 6, 1, 792, 85, 202, 134, 81, 634, 424, 41, 636, 191, 79, 348, 131, 27, 263, 2742, 30, 2, 2, 17, 9, 142, 46, 1782, 4601, 4185, 178, 346, 3304, 21, 390, 358, 2, 7, 67, 528, 2430, 68, 1431, 2054, 321, 24, 990, 799, 357, 318, 2080, 2597, 264, 1205, 9, 6128, 6, 9, 349, 384, 896, 107, 770, 10, 3, 84, 169, 153, 120, 9, 63, 191, 7], [115, 234, 232, 1357, 410, 433, 308, 741, 2071, 792, 459, 295, 58, 93, 186, 145, 317, 2387, 93, 319, 31, 147, 1, 2, 81, 2511, 116, 45, 903, 102, 44, 21, 1385, 45, 931, 91, 936, 93, 2387, 3016, 317, 1074, 55, 15, 3634, 1646, 1186, 541, 2, 31, 319, 1207, 3016, 929, 93, 6, 231, 1518, 3635, 4602, 1146, 3016, 1208, 82, 317, 530, 771, 7479, 661, 93, 1482, 317, 116, 2387, 980, 166, 2404, 116, 5583, 317, 45, 261, 1453, 3101, 3584, 1146, 530, 35, 82, 5749, 1967, 530, 3202, 534, 334, 981, 10353, 5, 3698, 2765, 34, 111, 45, 4, 15, 362, 642, 12760, 91, 1, 32, 15, 362, 459, 126, 57, 262, 1, 94, 3346, 85, 202, 7, 496, 1255, 281, 25427, 2003, 340, 167, 2

In [20]:
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [21]:
X.shape

(20491, 120)

In [22]:
glove_vectors = dict()

In [23]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-06-18 13:37:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-18 13:37:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-18 13:37:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [24]:
file = open('/content/glove.6B.200d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    #storing the word in the variable
    vectors = np.asarray(values[1: ])
    #storing the vector representation of the respective word in the dictionary
    glove_vectors[word] = vectors
file.close()

In [25]:
len(glove_vectors)

400000

In [26]:
keys = glove_vectors.keys()
len(keys)

400000

In [27]:
word_vector_matrix = np.zeros((vocab_size, 200))

for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    else:
      print(word)

Streaming output truncated to the last 5000 lines.
restautant
luisander
cleaningcutting
grasspaintingand
thanksmark
payogan
realityas
specia
agun
dboutique
seminayak
playwalk
abosolutely
dissopointed
streetbeach
iceextra
crystalised
midlow
shoppingrestauranthotel
waikikibased
slowon
comptuer
guessi
etcmark
flamboyan
topocia
incredibledo
toursim
enjoyablein
aquafrenzy
avenguda
drassanes
eitherback
aroundreception
noisyroom
coldyeah
quietwhat
loungebreakfastsnack
camperthank
areacome
roomscasa
barcelonain
pleasejames
amazningly
starsfirst
cigarettereeking
comerc
muchconclusion
receivedit
rateit
travellerhighly
roombedroom
teascoffee
inbesides
quitetthe
corridorfinally
strenous
ensimadas
implimented
lobbysome
tastestaff
mulitlingual
childern
llet
sanduich
mnham
attitud
safesome
oldwhat
moisy
aerobed
airportkeep
petalswe
podlike
overbearingthank
hatbour
agaim
momentwe
wehave
arabsthe
fooddelivered
everytimehaving
conveniencefor
effortlesswe
incrediblypolite
ladyindeed
seasonsbook
superlati

In [28]:
word_vector_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3, stratify = y)

In [30]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [31]:
vec_size = 200

model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights = [word_vector_matrix], trainable = False))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.3))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.3))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy',f1_m,precision_m,recall_m])

model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
449/449 [==============================] - 40s 83ms/step - loss: 0.4016 - accuracy: 0.8399 - f1_m: 0.9101 - precision_m: 0.8453 - recall_m: 0.9920 - val_loss: 0.3121 - val_accuracy: 0.8684 - val_f1_m: 0.9260 - val_precision_m: 0.8728 - val_recall_m: 0.9887
Epoch 2/10
449/449 [==============================] - 36s 81ms/step - loss: 0.2797 - accuracy: 0.8855 - f1_m: 0.9331 - precision_m: 0.9060 - recall_m: 0.9647 - val_loss: 0.2599 - val_accuracy: 0.9029 - val_f1_m: 0.9433 - val_precision_m: 0.9197 - val_recall_m: 0.9703
Epoch 3/10
449/449 [==============================] - 37s 81ms/step - loss: 0.2508 - accuracy: 0.9000 - f1_m: 0.9410 - precision_m: 0.9231 - recall_m: 0.9622 - val_loss: 0.2510 - val_accuracy: 0.9065 - val_f1_m: 0.9456 - val_precision_m: 0.9181 - val_recall_m: 0.9766
Epoch 4/10
449/449 [==============================] - 37s 82ms/step - loss: 0.2339 - accuracy: 0.9048 - f1_m: 0.9435 - precision_m: 0.9277 - recall_m: 0.9622 - val_loss: 0.2438 - val_accuracy: 0.9

In [32]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.9102147221565247:
f1_score = 0.947448194026947
Precision = 0.9261271953582764
Recall = 0.9714908003807068
